In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import sys, os

from scipy.sparse.linalg import eigsh
from scipy.sparse import diags_array

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from notebook_functions import load_data

In [ ]:
def convg_plot(path, data_str, lfc_degrees_and_markers, lower_limit=None, upper_limit=None, leapfrog_on_coarse_cfl=None):
    if leapfrog_on_coarse_cfl:
        plt.vlines(leapfrog_on_coarse_cfl, lower_limit, upper_limit, linestyles='dashed', colors='k')

    data = pd.read_table(path + data_str + '.txt', sep='|')
    data.drop(columns=data.columns[0], axis=1, inplace=True)
    data.columns = data.columns.str.strip()
        
    stepsizes = data['time step width'].unique()
    data['L2 error'] = data['L2 error'].apply(
        lambda x: 1e20 if isinstance(x, str) and x.strip() in ('nan', '-nan') else x
    ).astype(float)

    mask_large_vals = data['L2 error'] >= 100
    data.loc[mask_large_vals, 'L2 error'] = 1e20

    lfc_degrees = lfc_degrees_and_markers.keys()

    for lfc_degree, color in zip(lfc_degrees, list(mcolors.TABLEAU_COLORS)):
        mask_lfc_degree = data['lfc degree'] == lfc_degree
        errors = data.loc[mask_lfc_degree, 'L2 error']

        if lfc_degree == 1:
            label = "leapfrog"
        else:
            label = r"$p = $" + str(lfc_degree)
        plt.loglog(stepsizes, errors, marker=lfc_degrees_and_markers[lfc_degree], linestyle='-',color=color, label=label)

    # order line
    ord_2_line = 0.5 * errors.values[-1] / stepsizes[-1]**2
    plt.loglog(stepsizes, ord_2_line*stepsizes**2, linestyle='--', color='gray', label=r'$\mathcal{O}(\tau^2)$')
    
    plt.legend(loc='upper left')
    plt.ylabel(r"$L^2$-error")
    plt.xlabel(r"time stepsize $\tau$")
    if lower_limit and upper_limit: plt.ylim((lower_limit, upper_limit))
    plt.tight_layout()
    plt.show()

# Time convergence

In [ ]:
matrix_path = "../../build/TE-example/convergence/matrices/"
plot_data_path = "../../build/TE-example/convergence/"

localref = 3
globalref = 3
dg = 5
threshold_global_to_local = 1.2
eta = 1

data_str = f"_dg{dg}_globalref{globalref}_localref{localref}"

In [ ]:
indH, indE, mass_H, mass_E, inv_mass_H, inv_mass_E, curl_H, curl_E = load_data(matrix_path, data_str)
cutoff_nonstiff = diags_array(np.concatenate((np.ones(indH[2]), np.zeros(indH[4] - indH[2])), axis=None), format='csr')
nonstiff_curl_curl = -curl_H @ inv_mass_H @ cutoff_nonstiff @ curl_E
vals_nonstiff, vecs = eigsh(A=nonstiff_curl_curl, M=mass_E, Minv=inv_mass_E, k=3, which='LM')

In [ ]:
lfc_degrees_and_markers = {1 : '.', 2 : '.', 4 : 'X', 8 : 'x', 9 : '+'}
data_str = f"errors_time_cavity_LTS_LFC_eta{eta}" + data_str + f"_threshold{threshold_global_to_local}"

convg_plot(plot_data_path, data_str.replace('.', ','), lfc_degrees_and_markers=lfc_degrees_and_markers, lower_limit=3e-7, upper_limit=3e-5, leapfrog_on_coarse_cfl=np.sqrt(4 / np.max(vals_nonstiff)))

# Space-Time convergence

In [ ]:
plot_data_path = "../../build/TE-example/convergence/"

localref = 2
dg = 2
threshold_global_to_local = 1.2
eta = 1

data_str = f"errors_space_time_cavity_LTS_LFC_eta{eta}_dg{dg}_localref{localref}_threshold{threshold_global_to_local}".replace('.', ',')

In [ ]:
lfc_degrees_and_markers = {2 : '.', 4 : 'X', 8 : 'x', 9 : '+'}

convg_plot(plot_data_path, data_str, lfc_degrees_and_markers=lfc_degrees_and_markers)